A decision tree is a machine learning algorithm that can be used for classification or regression tasks. It works by creating a tree-like model of decisions and their possible consequences, with the goal of accurately predicting the outcome of a new data point based on its characteristics.

Here is an example of how you could implement a decision tree from scratch in Python:

In [3]:
class Node:
    def __init__(self, feature=None, split_point=None, left=None, right=None):
        self.feature = feature
        self.split_point = split_point
        self.left = left
        self.right = right

class DecisionTree:
    def __init__(self, criterion='gini'):
        self.criterion = criterion
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y):
        # base case: no more features or all samples belong to the same class
        if len(set(y)) == 1 or len(X[0]) == 0:
            leaf = Node(y)
            return leaf

        # find the best feature and split point
        best_feature, split_point = self._find_best_split(X, y)

        # split the data into two sets based on the best feature and split point
        X1, y1, X2, y2 = self._split_data(X, y, best_feature, split_point)

        # create a new internal node with the best feature and split point
        node = Node(best_feature, split_point)

        # recursive call to build the left and right subtrees
        node.left = self._build_tree(X1, y1)
        node.right = self._build_tree(X2, y2)

        return node

    def _find_best_split(self, X, y):
        # code to find the best feature and split point goes here
        pass

    def _split_data(self, X, y, feature, split_point):
        # code to split the data into two sets based on the given feature and split point goes here
        pass        


Note: This is just a high level representation of the process behind the original classifier in sklearn. There are still many details and considerations that need to be addressed, including handling missing values and dealing with overfitting.